# **NLP - Sentiment Analysis of Tweets using biLSTM**
1. [Dataset Preparation](#section1)
2. [Preprocessing](#section2)
3. [Model](#section3)
4. [Training](#section4)
5. [Prediction](#section5)

In [1]:
import csv
import pandas as pd
import numpy as np
import time
import spacy
import random
from pathlib import Path
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.legacy import data 
import torchtext
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [2]:
create_subset = False # set to false if sentiment140-small-tokenized.csv already exists

In [3]:
# Setting device on GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

torch.backends.cudnn.deterministic = True

Using device: cpu


<a id='section1'></a>
# **1. Dataset Preparation**
The first column contains the sentiments and the last column contains the tweets.

In [4]:
if create_subset:
    # Read in data into a dataframe
    df = pd.read_csv("training.1600000.processed.noemoticon.csv", engine="python", header=None)

    df.head(5)

The dataset consists of two sentiments (0 - negative, 4 - positive)

In [5]:
if create_subset:
    # Count the number of tweets per sentiment
    df[0].value_counts()

In [6]:
if create_subset:
    # Model the sentiments as binary (0 - negative, 1 - positive)
    df[0]=df[0].replace(to_replace=4,value=1)
    df[0].value_counts()

In [7]:
from sentiment_util import tokenize_csv

if create_subset:
    # Save a subset as a smaller dataset from training
    df.sample(500000).to_csv("sentiment140-small.csv", header=None, index=None)
    # remove stopwords, punctuation, and make everything lowercase
    tokenize_csv('sentiment140-small.csv', 'sentiment140-small-tokenized.csv')

[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>
[nltk_data] Error loading vader_lexicon: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>


<a id='section2'></a>
# **2. Preprocessing**

In [8]:
# Declare fields for tweets and labels
# include_lengths tells the RNN how long the actual sequences are
spacy.load('en_core_web_sm')

def tokenize(s):
    return s.split(' ')

TEXT = data.Field(tokenize=tokenize, lower=True, include_lengths=True)
UPVOTE = data.LabelField(sequential=False, use_vocab=False, dtype=torch.int64)
CHANGE = data.LabelField(sequential=False, use_vocab=False, dtype=torch.float)
SENT_LABEL = data.LabelField(sequential=False, dtype=torch.int64)
CHAN_LABEL = data.LabelField(sequential=False, dtype=torch.int64)

# Map data to fields
fields = [('text', TEXT), ('upvote', None), ('change', None), ('sent_label', SENT_LABEL), ('chan_label', None)]

# Apply field definition to create torch dataset
dataset = torchtext.legacy.data.TabularDataset(
        path="reddit_sentiments.csv",
        format="CSV",
        fields=fields,
        skip_header=False)

# Split data into train, test, validation sets
(train_data, test_data, valid_data) = dataset.split(split_ratio=[0.8,0.1,0.1])

print("Number of train data: {}".format(len(train_data)))
print("Number of test data: {}".format(len(test_data)))
print("Number of validation data: {}".format(len(valid_data)))

Number of train data: 145027
Number of test data: 18129
Number of validation data: 18128


In [9]:
# An example from the training set
print(vars(train_data.examples[10]))

{'text': ['right', 'mind', 'would', 'buy', 'nkla', 'today', ''], 'sent_label': '0'}


### **Build Vocabulary**
Build the vocabulary for the training set using pre-trained GloVe embeddings.
GloVe embeddings were trained on 6 billion tokens and the embeddings are 100-dimensional.

In [10]:
MAX_VOCAB_SIZE = 10000000

# unk_init initializes words in the vocab using the Gaussian distribution
TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.100d",
                 unk_init = torch.Tensor.normal_)

# build vocab for training set - convert words into integers
SENT_LABEL.build_vocab(train_data)

# Most frequent tokens
TEXT.vocab.freqs.most_common(10)

[('', 145027),
 ('gme', 23644),
 ('spy', 19824),
 ('call', 18813),
 ('aapl', 14599),
 ('nio', 13151),
 ('tesla', 11639),
 ('buy', 10403),
 ('today', 10361),
 ('go', 9765)]

### **Iterator**
Pad each tweet to be the same length to process in batch. 
The BucketIterator will group tweets of similar lengths together for minimized padding in each batch.


In [11]:
BATCH_SIZE = 128

# sort_within_batch sorts all the tensors within a batch by their lengths
train_iterator, valid_iterator, test_iterator = torchtext.legacy.data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    device = device,
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True)

<a id='section3'></a>
# **3. Model**

### **Create Model**

In [12]:
from models.sentiment_model import SentimentLSTM

INPUT_DIM = len(TEXT.vocab)
# dim must be equal to the dim of pre-trained GloVe vectors
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 3
# 2 layers of biLSTM
N_LAYERS = 2
BIDIRECTIONAL = True
# Dropout probability
DROPOUT = 0.5
# Get pad token index from vocab
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

# Create an instance of LSTM class
model = SentimentLSTM(INPUT_DIM,
            EMBEDDING_DIM,
            HIDDEN_DIM,
            OUTPUT_DIM,
            N_LAYERS,
            BIDIRECTIONAL,
            DROPOUT,
            PAD_IDX)

print(model)

SentimentLSTM(
  (embedding): Embedding(29194, 100, padding_idx=1)
  (encoder): LSTM(100, 256, num_layers=2, dropout=0.5, bidirectional=True)
  (predictor): Linear(in_features=512, out_features=3, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [13]:
# Sample from the training set
print(vars(train_iterator.dataset[10]))

{'text': ['right', 'mind', 'would', 'buy', 'nkla', 'today', ''], 'sent_label': '0'}


In [14]:
# Copy the pre-trained word embeddings into the embedding layer
pretrained_embeddings = TEXT.vocab.vectors

# [vocab size, embedding dim]
print(pretrained_embeddings.shape)

torch.Size([29194, 100])


In [15]:
# Replace the initial weights of the embedding layer with the pre-trained embeddings
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.5171,  1.0985, -0.2348,  ...,  1.3741,  0.5025,  0.5519],
        [-0.2804, -0.4799,  0.5140,  ..., -0.7849, -1.0795,  0.7072],
        [-1.0218, -0.3157,  1.0438,  ...,  1.1108, -0.0150, -0.2370],
        ...,
        [ 0.1762, -0.1919,  0.2605,  ...,  0.1858,  0.9024,  1.1511],
        [ 0.4593,  0.0251,  0.0319,  ..., -0.0136, -0.5006,  0.5798],
        [ 0.2011,  0.2284,  0.7082,  ...,  1.2060, -0.8663,  0.8965]])

In [16]:
# Initialize <unk> and <pad> both to all zeros - irrelevant for sentiment analysis
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

# Setting row in the embedding weights matrix to zero using the token index
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-1.0218, -0.3157,  1.0438,  ...,  1.1108, -0.0150, -0.2370],
        ...,
        [ 0.1762, -0.1919,  0.2605,  ...,  0.1858,  0.9024,  1.1511],
        [ 0.4593,  0.0251,  0.0319,  ..., -0.0136, -0.5006,  0.5798],
        [ 0.2011,  0.2284,  0.7082,  ...,  1.2060, -0.8663,  0.8965]])


<a id='section4'></a>
# **4. Training**

In [17]:
# Adam optimizer used to update the weights
# optimizer = optim.Adam(model.parameters(), lr=2e-3)

# Loss function: binary cross entropy with logits
# It restricts the predictions to a number between 0 and 1 using the logit function
# then use the bound scarlar to calculate the loss using binary cross entropy
criterion = nn.CrossEntropyLoss

# Use GPU
model = model.to(device)
criterion = criterion.to(device) 

AttributeError: 'torch.device' object has no attribute '_apply'

### **Train the model**

In [ ]:
def batch_accuracy(predictions, label):
    """
    Returns accuracy per batch.

    predictions - float
    label - 0 or 1
    """

    # Round predictions to the closest integer using the sigmoid function
    preds = torch.round(torch.sigmoid(predictions))
    # If prediction is equal to label
    correct = (preds == label).float()
    # Average correct predictions
    accuracy = correct.sum() / len(correct)

    return accuracy

def timer(start_time, end_time):
    """
    Returns the minutes and seconds.
    """

    time = end_time - start_time
    mins = int(time / 60)
    secs = int(time - (mins * 60))

    return mins, secs

In [ ]:
def train(model, iterator, optimizer, criterion):
    """
    Function to evaluate training loss and accuracy.

    iterator - train iterator
    """
    
    # Cumulated Training loss
    training_loss = 0.0
    # Cumulated Training accuracy
    training_acc = 0.0
    
    # Set model to training mode
    model.train()
    
    # For each batch in the training iterator
    for batch in iterator:
        
        # 1. Zero the gradients
        optimizer.zero_grad()
        
        # batch.text is a tuple (tensor, len of seq)
        text, text_lengths = batch.text
        
        # 2. Compute the predictions
        predictions = model(text, text_lengths).squeeze(1)
        
        
        # 3. Compute loss
        loss = criterion(predictions, batch.sent_labels)
        
        # Compute accuracy
        accuracy = batch_accuracy(predictions, batch.sent_labels)
        
        # 4. Use loss to compute gradients
        loss.backward()
        
        # 5. Use optimizer to take gradient step
        optimizer.step()
        
        training_loss += loss.item()
        training_acc += accuracy.item()
        
    # Return the loss and accuracy, averaged across each epoch
    # len of iterator = num of batches in the iterator
    return training_loss / len(iterator), training_acc / len(iterator)


def evaluate(model, iterator, criterion):
    """
    Function to evaluate the loss and accuracy of validation and test sets.

    iterator - validation or test iterator
    """
    
    # Cumulated Training loss
    eval_loss = 0.0
    # Cumulated Training accuracy
    eval_acc = 0
    
    # Set model to evaluation mode
    model.eval()
    
    # Don't calculate the gradients
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.sent_labels)
            
            accuracy = batch_accuracy(predictions, batch.sent_labels)

            eval_loss += loss.item()
            eval_acc += accuracy.item()
        
    return eval_loss / len(iterator), eval_acc / len(iterator)        

In [ ]:
# Number of epochs
NUM_EPOCHS = 10

# Lowest validation lost
best_valid_loss = float('inf')

learning_rates = [1e-3, 5e-3, 1e-2, 5e-2]

for lr in learning_rates:
    print(f'learning rate is {lr}')
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(NUM_EPOCHS):

        start_time = time.time()

        # Evaluate training loss and accuracy
        train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
        # Evaluate validation loss and accuracy
        valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

        end_time = time.time()

        mins, secs = timer(start_time, end_time)

        # At each epoch, if the validation loss is the best
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            # Save the parameters of the model
            torch.save(model.state_dict(), 'trained_sentiment.pt')

        print("Epoch {}:".format(epoch+1))
        print("\t Total Time: {}m {}s".format(mins, secs))
        print("\t Train Loss {} | Train Accuracy: {}%".format(round(train_loss, 2), round(train_acc*100, 2)))
        print("\t Validation Loss {} | Validation Accuracy: {}%".format(round(valid_loss, 2), round(valid_acc*100, 2)))

<a id='section5'></a>
# **5. Prediction**

In [ ]:
from utils.sentiment_util import predict

In [ ]:
# Load the model with the best validation loss
model.load_state_dict(torch.load('model-small.pt'))

# Evaluate test loss and accuracy
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print("Test Loss: {} | Test Acc: {}%".format(round(test_loss, 2), round(test_acc*100, 2)))